In [0]:
#importing some useful libraries
import time
import matplotlib
import matplotlib.pylab as plt
import seaborn as sns
import datetime
import math
import pickle
from sklearn.metrics import roc_curve
import numpy as np, pandas as pd, os, gc
from sklearn.model_selection import GroupKFold
from sklearn.metrics import confusion_matrix,roc_auc_score,roc_curve,auc
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import GridSearchCV,RandomizedSearchCV
import xgboost as xgb

import warnings
warnings.filterwarnings('ignore')

In [79]:
from google.colab import drive
drive.mount('/content/drive/')

Drive already mounted at /content/drive/; to attempt to forcibly remount, call drive.mount("/content/drive/", force_remount=True).


In [0]:
path = '/content/drive/My Drive/IEEE_F/'
# LOAD TRAIN
X_train = pd.read_csv(path + 'train_transaction.csv',index_col='TransactionID')
train_id = pd.read_csv(path + 'train_identity.csv',index_col='TransactionID')
X_train = X_train.merge(train_id, how='left', left_index=True, right_index=True)
# LOAD TEST
X_test = pd.read_csv(path + 'test_transaction.csv',index_col='TransactionID')
test_id = pd.read_csv(path + 'test_identity.csv',index_col='TransactionID')
X_test = X_test.merge(test_id, how='left', left_index=True, right_index=True)
# TARGET
y_train = X_train['isFraud'].copy()
del train_id, test_id, X_train['isFraud']; x = gc.collect()

In [0]:
def final_model(data):
  cat_type = ['ProductCD', 'card4', 'card6', 'P_emaildomain', 'R_emaildomain','M1', 'M2', 'M3', 'M4','M5','M6', 'M7', 'M8', 'M9', 'id_12', 'id_15', 'id_16', 'id_23', 'id_27', 'id_28', 'id_29', 'id_30', 'id_31', 'id_33', 'id_34', 'id_35', 'id_36', 'id_37', 'id_38', 'DeviceType', 'DeviceInfo']
  for col in data.columns:
    if col in cat_type:
      data[col] = data[col].astype('category')
      X_test[col] = X_test[col].astype('category')
    else:
      data[col] = data[col].astype('float32')
      X_test[col] = X_test[col].astype('float32')
    
  for i,f in enumerate(data.columns):
    if f in cat_type : 
      df_comb = pd.concat([data[f],X_test[f]],axis=0)
      df_comb,_ = df_comb.factorize(sort=True)
      if df_comb.max()>32000: print(f,'needs int32')
      data[f] = df_comb[:len(data)].astype('int16')
      X_test[f] = df_comb[len(data):].astype('int16')
        
    elif f not in ['TransactionAmt','TransactionDT']:
      mn = data[f].min()
      data[f] -= np.float32(mn)
      X_test[f] -= np.float32(mn)
      data[f].fillna(-1,inplace=True)
      X_test[f].fillna(-1,inplace=True)
    
  for i in range(1,16):
    if i in [1,2,3,5,9]: continue
    data['D'+str(i)] =  data['D'+str(i)] - data.TransactionDT/np.float32(24*60*60)
    X_test['D'+str(i)] = X_test['D'+str(i)] - X_test.TransactionDT/np.float32(24*60*60)

    
  def freq_encode(df1, df2, cols):
    for col in cols:
      df = pd.concat([df1[col],df2[col]])
      vc = df.value_counts(dropna=True, normalize=True).to_dict() #creating a dict with feature value as key and feature vake count as value.
      vc[-1] = -1   #for nan values
      cn = col+'_FE'
      df1[cn] = df1[col].map(vc)  #mapping values according to feature value
      df1[cn] = df1[cn].astype('float32')   #storing in float 32 format
      df2[cn] = df2[col].map(vc)
      df2[cn] = df2[cn].astype('float32')
        
    
  def label_encode(col,train=data,test=X_test,verbose=True):
    df_comb = pd.concat([train[col],test[col]],axis=0)
    df_comb,_ = df_comb.factorize(sort=True)
    cn = col
    if df_comb.max()>32000:
      train[cn] = df_comb[:len(train)].astype('int32')
      test[cn] = df_comb[len(train):].astype('int32')
    else:
      train[cn] = df_comb[:len(train)].astype('int16')
      test[cn] = df_comb[len(train):].astype('int16')
    del df_comb; x=gc.collect()      
           
  def encode_AG(main_columns, uids, aggregations=['mean'], train_df=data, test_df=X_test, 
                fillna=True, usena=False):
    for main_column in main_columns:
      for col in uids:
        for agg_type in aggregations:
          new_col_name = main_column+'_'+col+'_'+agg_type
          temp_df = pd.concat([train_df[[col, main_column]], test_df[[col,main_column]]])
          if usena: temp_df.loc[temp_df[main_column]==-1,main_column] = np.nan
          temp_df = temp_df.groupby([col])[main_column].agg([agg_type]).reset_index().rename(columns={agg_type: new_col_name}) #taking aggregation according to group by col 
          temp_df.index = list(temp_df[col])
          temp_df = temp_df[new_col_name].to_dict()   

          train_df[new_col_name] = train_df[col].map(temp_df).astype('float32')
          test_df[new_col_name]  = test_df[col].map(temp_df).astype('float32')
                
          if fillna:
            train_df[new_col_name].fillna(-1,inplace=True)
            test_df[new_col_name].fillna(-1,inplace=True)
            
                
  def comb_ftr(col1,col2,df1=data,df2=X_test):
    cn = col1+'_'+col2
    df1[cn] = df1[col1].astype(str)+'_'+df1[col2].astype(str) # combining two feature value with _ as sepration
    df2[cn] = df2[col1].astype(str)+'_'+df2[col2].astype(str) 
    label_encode(cn,verbose=False)
      
    
  def AG_enc_nunq(main_columns, uids, train_df=data, test_df=X_test):
    for main_column in main_columns:
      for col in uids:
        comb = pd.concat([train_df[[col]+[main_column]],test_df[[col]+[main_column]]],axis=0)
        mp = comb.groupby(col)[main_column].agg(['nunique'])['nunique'].to_dict()
        train_df[col+'_'+main_column+'_ct'] = train_df[col].map(mp).astype('float32')
        test_df[col+'_'+main_column+'_ct'] = test_df[col].map(mp).astype('float32')
          
    
  data['P_isproton']=(data['P_emaildomain']=='protonmail.com')
  data['R_isproton']=(data['R_emaildomain']=='protonmail.com')
  data['cents'] = (data['TransactionAmt'] - np.floor(data['TransactionAmt'])).astype('float32')
  X_test['P_isproton']=(X_test['P_emaildomain']=='protonmail.com')
  X_test['R_isproton']=(X_test['R_emaildomain']=='protonmail.com')
  X_test['cents'] = (X_test['TransactionAmt'] - np.floor(data['TransactionAmt'])).astype('float32')
  freq_encode(data,X_test,['addr1','card1','card2','card3','P_emaildomain'])
  comb_ftr('card1','addr1')
  comb_ftr('card1_addr1','P_emaildomain')
  freq_encode(data,X_test,['card1_addr1','card1_addr1_P_emaildomain'])
  encode_AG(['TransactionAmt','D9','D11'],['card1','card1_addr1','card1_addr1_P_emaildomain'],['mean','std'],usena=True)
  START_DATE = datetime.datetime.strptime('2017-11-30', '%Y-%m-%d')
  data['DT_M'] = data['TransactionDT'].apply(lambda x: (START_DATE + datetime.timedelta(seconds = x)))
  data['DT_M'] = (data['DT_M'].dt.year-2017)*12 + data['DT_M'].dt.month 
  X_test['DT_M'] = X_test['TransactionDT'].apply(lambda x: (START_DATE + datetime.timedelta(seconds = x)))
  X_test['DT_M'] = (X_test['DT_M'].dt.year-2017)*12 + X_test['DT_M'].dt.month

  data['day'] = data.TransactionDT / (24*60*60)
  data['uid'] = data.card1_addr1.astype(str)+'_'+np.floor(data.day-data.D1).astype(str)

  X_test['day'] = X_test.TransactionDT / (24*60*60)
  X_test['uid'] = X_test.card1_addr1.astype(str)+'_'+np.floor(X_test.day-X_test.D1).astype(str)

    # FREQUENCY ENCODE UID
  freq_encode(data,X_test,['uid'])
    # AGGREGATE 
  encode_AG(['TransactionAmt','D4','D9','D10','D15'],['uid'],['mean','std'],fillna=True,usena=True)
    # AGGREGATE
  encode_AG(['C'+str(x) for x in range(1,15) if x!=3],['uid'],['mean'],data,X_test,fillna=True,usena=True)
    # AGGREGATE
  encode_AG(['M'+str(x) for x in range(1,10)],['uid'],['mean'],fillna=True,usena=True)
    # AGGREGATE
  AG_enc_nunq(['P_emaildomain','dist1','DT_M','id_02','cents'], ['uid'], train_df=data, test_df=X_test)
    # AGGREGATE
  encode_AG(['C14'],['uid'],['std'],data,X_test,fillna=True,usena=True)
    # AGGREGATE 
  AG_enc_nunq(['C13','V314'], ['uid'], train_df=data, test_df=X_test)
    # AGGREATE 
  AG_enc_nunq(['V127','V136','V309','V307','V320'], ['uid'], train_df=data, test_df=X_test)
    # NEW FEATURE
  data['outsider15'] = (np.abs(data.D1-data.D15)>3).astype('int8')
  rejected_variables = ['V2', 'V5', 'V7', 'V9', 'V10', 'V12', 'V15', 'V16', 'V18', 'V19', 'V21', 'V22', 'V24', 'V25', 'V28', 'V29', 'V31', 'V32', 'V33', 'V34', 'V35', 'V38', 'V39', 'V42', 'V43', 'V45', 'V46', 'V49', 'V50', 'V51', 'V52', 'V53', 'V55', 'V57', 'V58', 'V60', 'V61', 'V63', 'V64', 'V66', 'V69', 'V71', 'V72', 'V73', 'V74', 'V75', 'V77', 'V79', 'V81', 'V83', 'V84', 'V85', 'V87', 'V90', 'V92', 'V93', 'V94', 'V95', 'V96', 'V97', 'V98', 'V99', 'V100', 'V101', 'V102', 'V103', 'V104', 'V105', 'V106', 'V109', 'V110', 'V112', 'V113', 'V114', 'V116', 'V118', 'V119', 'V122', 'V125', 'V126', 'V128', 'V131', 'V132', 'V133', 'V134', 'V135', 'V137', 'V140', 'V141', 'V143', 'V144', 'V145', 'V146', 'V148', 'V149', 'V150', 'V151', 'V152', 'V153', 'V154', 'V155', 'V157', 'V158', 'V159', 'V161', 'V163', 'V164', 'V167', 'V168', 'V170', 'V172', 'V174', 'V177', 'V179', 'V181', 'V183', 'V184', 'V186', 'V189', 'V190', 'V191', 'V192', 'V193', 'V194', 'V195', 'V196', 'V197', 'V199', 'V200', 'V201', 'V202', 'V204', 'V206', 'V208', 'V211', 'V212', 'V213', 'V214', 'V216', 'V217', 'V219', 'V222', 'V225', 'V227', 'V230', 'V231', 'V232', 'V233', 'V236', 'V237', 'V239', 'V241', 'V242', 'V243', 'V244', 'V245', 'V246', 'V247', 'V248', 'V249', 'V251', 'V254', 'V255', 'V256', 'V259', 'V262', 'V263', 'V265', 'V268', 'V269', 'V270', 'V272', 'V273', 'V275', 'V276', 'V278', 'V279', 'V280', 'V282', 'V287', 'V288', 'V290', 'V292', 'V293', 'V295', 'V298', 'V299', 'V300', 'V302', 'V304', 'V306', 'V308', 'V311', 'V312', 'V313', 'V315', 'V316', 'V317', 'V318', 'V319', 'V321', 'V322', 'V323', 'V324', 'V325', 'V326', 'V327', 'V328', 'V329', 'V330', 'V331', 'V332', 'V333', 'V334', 'V335', 'V336', 'V337', 'V338', 'V339','D6','D7','D8','D9','D12','D13','D14','C3','M5','id_08','id_33','card4','id_07','id_14','id_21','id_30','id_32','id_34','id_22', 'id_23', 'id_24', 'id_25', 'id_26', 'id_27']
  drop_variables = rejected_variables + ['DT_M', 'TransactionDT', 'day', 'uid']
  cols = list( X_train.columns)
  for c in drop_variables:
    cols.remove(c)
  with open(path + 'clfxgbWcb.pkl', 'rb') as f:
    model = pickle.load(f)

  preds = model.predict_proba(data[cols].iloc[:442905])[:,1]
  auc_train = roc_auc_score(y_train[:442905],preds)
  preds = model.predict_proba(data[cols].iloc[442905:])[:,1]
  auc_val = roc_auc_score(y_train[442905:],preds)

  return auc_train,auc_val
    


In [85]:
a,b = final_model(data = X_train)
print('ROC_AUC train=',a)
print('ROC_AUC local validate=',b)

ROC_AUC train= 0.998904017818575
ROC_AUC local validate= 0.9891984080965321
